# module import

In [1]:
import pandas as pd
import numpy as np
import gc
import os
import warnings
warnings.filterwarnings("ignore")

from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

data_path = "../../3. 경서경욱_데이터 및 모델 세이브 파일/dataset/"

# Read data

In [2]:
pdde = pd.read_csv(data_path + "origin_dataset/LPOINT_BIG_COMP_02_PDDE.csv") # 유통사 상품 구매 내역
product = pd.read_csv(data_path + "origin_dataset/LPOINT_BIG_COMP_04_PD_CLAC.csv") # 유통사 상품 카테고리 마스터

# preprocessing

In [3]:
# 상품코드에 따른 상품명 labeling feature

pdde = pdde.merge(product, on = "pd_c", how = "left")
pdde = pdde.drop("pd_c", axis = 1)
pdde.head(3)

,cust,rct_no,chnl_dv,cop_c,br_c,de_dt,de_hr,buy_am,buy_ct,pd_nm,clac_hlv_nm,clac_mcls_nm
0,M430112881,A01000001113,1,A01,A010039,20210101,10,15000.0,1,남성티셔츠,남성의류,남성의류상의
1,M646853852,A01000002265,1,A01,A010025,20210101,10,79700.0,1,기타주방일회용품,주방잡화,주방일회용품
2,M430112881,A01000003148,1,A01,A010039,20210101,10,19000.0,1,남성티셔츠,남성의류,남성의류상의


# 장바구니 분석을 통한 상품추천
## - class 설명 -
#### def __init__()
    1) 초기값으로 상품구매데이터 : self.data
    
    2) def make_basket()
        -> 입력된 data로 장바구니(basket)생성
          * 같은 상품대분류(clac_hlv_nm)에 속한 상품들끼리의 집합
          * 같은 영수증 번호를 가진 상품 (같은 결제 내역에 포함된 상품)
        -> 같은 영수증번호에 있는 상품들 중 같은 대분류에 해당하는 상품들끼리 장바구니 생성
        
    3) def make_support_score()
        -> make_basket()에서 생성된 장바구니(basket)을 통해 장바구니 분석 진행
        -> 지지도와 해당하는 장바구니를 return
        
    4) make_basket으로 장바구니 초깃값 생성 : self.basket
    5) make_support_score로 초깃값 생성 : self.support
 
#### def pro_recommend()
    1) 초깃값에서 생성된 support_score를 바탕으로 입력된 product와 지지도가 형성된 상품 추천

In [4]:
class support():
    def __init__(self, data):
        self.data = data

        def make_basket(self):
            clacs_ = list(self.data.clac_mcls_nm.unique())
            basket = pd.DataFrame()
            for clac in clacs_:
                df = self.data.query("clac_mcls_nm == @clac & clac_hlv_nm != '담배'").groupby(
                    ['cust', "de_dt", "de_hr", "cop_c"]).pd_nm.apply(lambda x : list(x)).reset_index()
                df["pd_nm"] = df["pd_nm"].apply(lambda x : list(set(x)))
                df["cnt"] = df["pd_nm"].apply(lambda x : len(x))
                df = df.query("cnt > 1")
                df["clac_mcls_nm"] = clac
                basket = pd.concat([basket, df])
            return basket

        def make_support_score(self, basket):
            support = pd.DataFrame()
            for clac in basket.clac_mcls_nm.unique():
                lst = []
                for j in basket.query("clac_mcls_nm == @clac").pd_nm.values:
                    lst.append(j)
                te = TransactionEncoder()
                te_ary = te.fit_transform(lst)
                df = pd.DataFrame(te_ary, columns=te.columns_)
                frequent_itemsets = apriori(df, min_support=0.2, use_colnames=True)
                frequent_itemsets["clac_mcls_nm"] = clac
                support = pd.concat([support, frequent_itemsets])
                support["cnt"] = support["itemsets"].apply(lambda x : len(list(x)))
                support = support.query("cnt > 1")
                support["itemsets"] = support["itemsets"].apply(lambda x : list(x))
            return support
        
        self.basket = make_basket(self)
        self.support = make_support_score(self, self.basket)
    
    def pro_recommend(self, product):
        lst_ = []
        for i in range(self.support.shape[0]):
            if product in self.support.iloc[i, 1]:
                lst_.append(i)
        pro_ = []
        for i in self.support.iloc[lst_, 1].values:
            pr = i.copy(); pr.remove(product)
            pro_.extend(pr)
        print(pro_, "를 추천드립니다")

In [5]:
if __name__ == "__main__":
    product = "남성티셔츠"
    recommend_class = support(pdde)
    recommend_class.pro_recommend(product)

['남성남방셔츠'] 를 추천드립니다
